# Tensorflow Transformer Pipeline with LTL Synthesis Data

In [ ]:
import tensorflow as tf

from ml2.aiger import AIGERCircuit, AIGERToSeqTokenizer
from ml2.datasets import CSVDataset
from ml2.ltl import DecompLTLSpec, LTLSpec
from ml2.ltl.ltl_spec import LTLSpecToSeqTokenizer
from ml2.pipelines import TFTransformerPipeline
from ml2.pipelines.tf_transformer_pipeline import TFTransformerConfig

## Tokenizer

### Input Tokenizer

In [ ]:
input_tokenizer = LTLSpecToSeqTokenizer(pad=64)

In [ ]:
spec_data = CSVDataset.load('scpa-2/val', 'ltl-syn', dtype=DecompLTLSpec)
input_tokenizer.build_vocabulary(spec_data.generator())


### Target Tokenizer

In [ ]:
target_tokenizer = AIGERToSeqTokenizer(eos=True, pad=64)

In [ ]:
circuit_data = CSVDataset.load('scpa-2/val', 'ltl-syn', dtype=AIGERCircuit)
target_tokenizer.build_vocabulary(circuit_data.generator(), add_start=True)

## Pipeline

In [ ]:
model_config = TFTransformerConfig(
  alpha=0.5,
  beam_size=2,
  d_embed_dec=128,
  d_embed_enc=128,
  d_ff=512,
  dropout=0.0,
  dtype_float=tf.float32,
  dtype_int=tf.int32,
  ff_activation="relu",
  num_heads=4,
  num_layers_dec=4,
  num_layers_enc=4,
)

In [ ]:
pipe = TFTransformerPipeline(
    name='tf-transformer-pipe-0',
    model_config=model_config,
    project='ltl-syn',
    input_tokenizer=input_tokenizer,
    target_tokenizer=target_tokenizer,
    max_input_length=64,
    max_target_length=64,
)

In [ ]:
spec = LTLSpec.from_dict({'formula': 'G (i0 -> F o0)', 'inputs': ['i0'], 'outputs': ['o0']})

In [ ]:
pipe.eval_sample(spec)